# Etapas de Pré-Processamento de Dados

Este documento reúne todas as etapas realizadas no processo de pré-processamento dos dados, com foco na **limpeza** e preparação para análise.

Em caso de dúvidas, entre em contato com **André**, **Arthur** ou **Ju**.


## Passo 1: Importação e Visão Geral Inicial dos Dados

Nesta primeira etapa, importamos o conjunto de dados Alunos_Atendimentos_Merged.csv utilizando a biblioteca pandas e realizamos uma inspeção inicial. A exibição das primeiras cinco linhas (`df.head()`) e dos tipos de dados de cada coluna (`df.dtypes`) nos permite ter uma compreensão preliminar da estrutura e do conteúdo do DataFrame.

In [78]:
import os
import pandas as pd

# --- Definição do Arquivo ---
file_path = '../data/interim/atendimentos_de_alunos.csv'

# --- Checagem de Existência do Arquivo ---
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Arquivo não encontrado: {file_path}")

# --- Leitura ---
try:
    df = pd.read_csv(file_path, sep=';', encoding='utf-8')

except pd.errors.ParserError as e:
    raise ValueError(f"Erro ao processar o CSV: {e}")

except Exception as e:
    raise RuntimeError(f"Ocorreu um erro inesperado ao ler o arquivo: {e}")

# --- Preview do DataFrame ---
df.head()

,Grupo % Cursado,Grupo_Acesso,MATRICULAID,DATAMATRICULA,ENCERRAMENTO_CONTRATO,NOME CURSO PADRÃO,Situação Contrato,Documentos Pessoais Pendentes,SITUACAO,Status_Cliente,...,Ouvidoria,Problema Técnico,Processos Secretaria,Reclame aqui,Rematrícula,Retenção,Solicitação de documentos,Suporte de Acesso,Suporte Pedagogico,has_contact
0,01 - 0% Cursado,00 - Sem Acesso,853579.0,03/01/2024,8/23/2025,TÉCNICO EM PRODUÇÃO DE MATERIAIS BILINGUE EM L...,Encerrado,NaN,CANCELADO,Quitado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIM
1,01 - 0% Cursado,05 - Acima de 180 Dias,858262.0,7/22/2024,1/18/2025,TÉCNICO EM SEGURANÇA DO TRABALHO,Encerrado,"Fotos 3 X 4, Certidão de Nascimento ou Casamento",CANCELADO,Quitado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIM
2,01 - 0% Cursado,00 - Sem Acesso,858874.0,08/02/2024,08/02/2025,TÉCNICO EM RECURSOS HUMANOS,Encerrado,NaN,CANCELADO,Quitado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIM
3,01 - 0% Cursado,00 - Sem Acesso,858881.0,08/02/2024,08/02/2025,TÉCNICO EM RECURSOS HUMANOS,Vigente,NaN,CURSANDO,Quitado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIM
4,01 - 0% Cursado,00 - Sem Acesso,858926.0,08/05/2024,08/05/2025,TÉCNICO EM RECURSOS HUMANOS,Encerrado,"Comprovante de Endereço, Certidão de Nasciment...",CANCELADO,Quitado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIM


In [79]:
# --- Tipos de Dados das Colunas ---
df.dtypes.to_frame(name='Data Type')

,Data Type
Grupo % Cursado,object
Grupo_Acesso,object
MATRICULAID,float64
DATAMATRICULA,object
ENCERRAMENTO_CONTRATO,object
NOME CURSO PADRÃO,object
Situação Contrato,object
Documentos Pessoais Pendentes,object
SITUACAO,object
Status_Cliente,object


## Passo 2: Tratamento de Valores Ausentes e Dados Inconsistentes
Abaixo são exibidos os totais de valores nulos por coluna no dataset.  
Essa visualização ajuda a identificar rapidamente quais variáveis precisam de tratamento ou podem indicar inconsistências nos dados.

In [80]:
null_counts = df.isnull().sum()
null_counts = null_counts[null_counts > 0].sort_values(ascending=False)

# --- Mostra em DataFrame o número de valores ausentes na tabela
pd.DataFrame(null_counts, columns=['Valores Ausentes'])

,Valores Ausentes
Ouvidoria,17393
Estágio,17390
Processos Secretaria,17388
Suporte Pedagogico,17378
Rematrícula,17377
Correção - Plataforma,17369
Correção cadastral,17337
Diploma,17332
Reclame aqui,17310
Retenção,17300


#### Função de Preenchimentos de Nulos
A função `fill_missing_values` a seguir preenche os valores nulos.
- Valores Categóricos recebem `'Não Informado'`.
- Valores Quantitativos recevem `0`, como é observado no *dataset*.

In [ ]:
import pandas as pd

def fill_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preenche valores ausentes (NaN) em colunas específicas de um DataFrame.

    Esta função aplica estratégias de imputação baseadas no tipo de dado inferido
    e no contexto da informação, usando 'Não Informado' para colunas categóricas/objeto
    e 0 para colunas numéricas, para indicar a ausência de ocorrência.
    """

    # ID de Matrícula não deve ser Nulo e apresenta apenas um NaN nos dados. Opcionalmente, remover.
    df['MATRICULAID'] = df['MATRICULAID'].fillna(-1)

    # --- 1. Preenchimento de Colunas Categóricas/Texto ---
    df['Grupo % Cursado'] = df['Grupo % Cursado'].fillna('Não Informado')
    df['Grupo_Acesso'] = df['Grupo_Acesso'].fillna('Não Informado')
    df['DATAMATRICULA'] = df['DATAMATRICULA'].fillna('Não Informado')
    df['ENCERRAMENTO_CONTRATO'] = df['ENCERRAMENTO_CONTRATO'].fillna('Não Informado')
    df['NOME CURSO PADRÃO'] = df['NOME CURSO PADRÃO'].fillna('Não Informado')
    df['Situação Contrato'] = df['Situação Contrato'].fillna('Não Informado')
    df['Documentos Pessoais Pendentes'] = df['Documentos Pessoais Pendentes'].fillna('Não Informado')
    df['SITUACAO'] = df['SITUACAO'].fillna('Não Informado')
    df['Status_Cliente'] = df['Status_Cliente'].fillna('Não Informado')
    df['Data de nascimento'] = df['Data de nascimento'].fillna('Não Informado')
    df['fezPrimeiroAcesso'] = df['fezPrimeiroAcesso'].fillna('Não Informado')
    df['Forma de Pagamento Oficial'] = df['Forma de Pagamento Oficial'].fillna('Não Informado')
    df['ESTADO'] = df['ESTADO'].fillna('Não Informado')


    # --- 2. Tratamento de Colunas Numéricas com Preenchimento de String ---
    df['PercentualConclusao'] = df['PercentualConclusao'].fillna(0)
    df['% Docs Pessoais'] = df['% Docs Pessoais'].fillna('Não Informado')


    # --- 3. Preenchimento de Colunas Numéricas com 0 ---
    df['DisciplinasAprovadas'] = df['DisciplinasAprovadas'].fillna(0)
    df['DisciplinasTotais'] = df['DisciplinasTotais'].fillna(0)
    df['# parcelas Vencidas'] = df['# parcelas Vencidas'].fillna(0)


    # --- 4. Preenchimento de Colunas Relacionadas a Atendimentos e Outras Métricas com 0 ---
    df['Total _Atendimentos'] = df['Total _Atendimentos'].fillna(0)
    df['Acesso ao Portal'] = df['Acesso ao Portal'].fillna(0)
    df['Anexar Documentos'] = df['Anexar Documentos'].fillna(0)
    df['Apoio Pedagogico'] = df['Apoio Pedagogico'].fillna(0)
    df['Bot de Atendimento'] = df['Bot de Atendimento'].fillna(0)
    df['Contato Via Ligação'] = df['Contato Via Ligação'].fillna(0)
    df['Correção - Plataforma'] = df['Correção - Plataforma'].fillna(0)
    df['Correção cadastral'] = df['Correção cadastral'].fillna(0)
    df['Diploma'] = df['Diploma'].fillna(0)
    df['Disparos'] = df['Disparos'].fillna(0)
    df['Duvidas Gerais'] = df['Duvidas Gerais'].fillna(0)
    df['Erro'] = df['Erro'].fillna(0)
    df['Estágio'] = df['Estágio'].fillna(0)
    df['Financeiro'] = df['Financeiro'].fillna(0)
    df['Informações Comercias'] = df['Informações Comercias'].fillna(0)
    df['Onboarding'] = df['Onboarding'].fillna(0)
    df['Outros Atendimentos'] = df['Outros Atendimentos'].fillna(0)
    df['Ouvidoria'] = df['Ouvidoria'].fillna(0)
    df['Problema Técnico'] = df['Problema Técnico'].fillna(0)
    df['Processos Secretaria'] = df['Processos Secretaria'].fillna(0)
    df['Reclame aqui'] = df['Reclame aqui'].fillna(0)
    df['Rematrícula'] = df['Rematrícula'].fillna(0)
    df['Retenção'] = df['Retenção'].fillna(0)
    df['Solicitação de documentos'] = df['Solicitação de documentos'].fillna(0)
    df['Suporte de Acesso'] = df['Suporte de Acesso'].fillna(0)
    df['Suporte Pedagogico'] = df['Suporte Pedagogico'].fillna(0)

    return df

In [82]:
fill_missing_values(df)

null_counts = df.isnull().sum()
null_counts = null_counts[null_counts > 0].sort_values(ascending=False)

# --- Mostra em DataFrame o número de valores ausentes na tabela
pd.DataFrame(null_counts, columns=['Valores Ausentes'])

,Valores Ausentes
Total _Atendimentos,3060


In [71]:
df.drop_duplicates(inplace=True)

Aqui eu fiz para remover colunas duplicadas caso houvesse alguma.

In [62]:
df['DATAMATRICULA'] = pd.to_datetime(df['DATAMATRICULA'], errors='coerce')
df['ENCERRAMENTO_CONTRATO'] = pd.to_datetime(df['ENCERRAMENTO_CONTRATO'], errors='coerce')
df['Data de nascimento'] = pd.to_datetime(df['Data de nascimento'], errors='coerce')
print(df['Data de nascimento'])

#É esperado a "resposta de erro"

0              NaT
1              NaT
2       2005-02-02
3       2005-02-02
4       2005-02-02
           ...    
17389          NaT
17390          NaT
17391          NaT
17392          NaT
17393          NaT
Name: Data de nascimento, Length: 17164, dtype: datetime64[ns]


<ipython-input-62-a51927758e7b>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Data de nascimento'] = pd.to_datetime(df['Data de nascimento'], errors='coerce')


In [63]:
print("Valores únicos na coluna 'SITUACAO':")
print(df['SITUACAO'].unique())

#Possiveis respostas para realizar a verificação antes do mapeamento de Churn

Valores únicos na coluna 'SITUACAO':
['CANCELADO' 'CURSANDO' 'EVADIDO' 'FORMADO' 'CONCLUIDO_REPROVADO'
 'CONCLUIDO' 'Não Informado']


In [64]:
print("Valores únicos na coluna 'SITUACAO':")
print(df['SITUACAO'].unique())
print("-" * 30)

mapeamento_churn = {
    'CONCLUIDO_REPROVADO': 1,
    'CANCELADO': 1,
    'EVADIDO': 1,
    'Não Informado': 0,
    'CONCLUIDO': 0,
    'CURSANDO': 0,
    'FORMADO': 0
}

df['churn'] = df['SITUACAO'].map(mapeamento_churn)

print("Contagem de valores na nova coluna 'churn' (incluindo nulos):")
print(df['churn'].value_counts(dropna=False))
print("-" * 30)

df_modelo = df.dropna(subset=['churn']).copy()

df_modelo['churn'] = df_modelo['churn'].astype(int)

print("Contagem de valores na coluna 'churn' final (após limpeza):")
print(df_modelo['churn'].value_counts())
print("\nTamanho do DataFrame original:", df.shape)
print("Tamanho do DataFrame para o modelo:", df_modelo.shape)

print("\nExemplo do DataFrame final:")
print(df_modelo[['SITUACAO', 'churn']])


Valores únicos na coluna 'SITUACAO':
['CANCELADO' 'CURSANDO' 'EVADIDO' 'FORMADO' 'CONCLUIDO_REPROVADO'
 'CONCLUIDO' 'Não Informado']
------------------------------
Contagem de valores na nova coluna 'churn' (incluindo nulos):
churn
1    9445
0    7719
Name: count, dtype: int64
------------------------------
Contagem de valores na coluna 'churn' final (após limpeza):
churn
1    9445
0    7719
Name: count, dtype: int64

Tamanho do DataFrame original: (17164, 52)
Tamanho do DataFrame para o modelo: (17164, 52)

Exemplo do DataFrame final:
        SITUACAO  churn
0      CANCELADO      1
1      CANCELADO      1
2      CANCELADO      1
3       CURSANDO      0
4      CANCELADO      1
...          ...    ...
17389    EVADIDO      1
17390    EVADIDO      1
17391    EVADIDO      1
17392    EVADIDO      1
17393   CURSANDO      0

[17164 rows x 2 columns]


Aqui eu separei as diversas situações em grupos binários, onde caso algum deles tenha relação de ter finalizado o curso/estar finalizando/ não informado ser 0, enquanto aqueles que tão relacionados a não conclusão está definido como 1. Isso será útil para ajudar o código a se guiar baseado nas nossas necessidades.